In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

import geopandas as gpd
from eodag import EODataAccessGateway
import rioxarray as riox
import numpy as np
import folium
from folium import LayerControl
from eo_tools_dev.util import show_cog

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
log = logging.getLogger(__name__)

## Set up parameters and output dir

### Important
- The products must be **unzipped** either manually or by using `extract=True` in the download call. 
- In the following products, only IW1 and IW2 are relevant as IW3 contains only water pixels.

In [ ]:
# change to your custom locations
data_dir = "/data/S1"

# ids = [
#     'S1A_IW_SLC__1SDV_20240903T050122_20240903T050149_055496_06C55B_C466',
#     'S1A_IW_SLC__1SDV_20240915T050123_20240915T050150_055671_06CC45_8AD7']
ids = ['S1A_IW_SLC__1SDV_20240904T165208_20240904T165235_055518_06C62B_530B', 'S1A_IW_SLC__1SDV_20240916T165209_20240916T165236_055693_06CD1B_B4BB']
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
outputs_prefix="/data/res/flood-poland-2024"

## Download S-1 products

In [ ]:
# load a geometry
file_aoi = "/data/South_Poland.geojson"
shp = gpd.read_file(file_aoi).geometry[0]

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2024-09-01",
    "end": "2024-09-17",
    "geom": shp
}

results, _ = dag.search(**search_criteria)
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, outputs_prefix="/data/S1/", extract=False)

## Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import prepare_insar

out_dir = prepare_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="full",
    subswaths=["IW1", "IW2", "IW3"],
    cal_type="sigma",
    apply_fast_esd=False,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    warp_kernel="bicubic",
)

## Define a simple amplitude change detection function

In [ ]:
def change_detection(file_amp_prm, file_amp_sec, file_out):
    from ndsar import sarblf
    amp_prm = riox.open_rasterio(file_amp_prm)[0]#.rolling(x=5, y=5, center=True).mean()
    amp_sec = riox.open_rasterio(file_amp_sec)[0]#.rolling(x=5, y=5, center=True).mean()
    # flt = amp_prm.data.copy()
    # log.info("Smoothing amplitudes")
    # amp_prm.data = sarblf(amp_prm.data, 1.6, 0.8)
    # amp_sec.data = sarblf(amp_sec.data, 1.6, 0.8)
    log.info("Incoherent changes")
    ch = np.log(amp_prm+1e-10) - np.log(amp_sec+1e-10)
    ch.rio.to_raster(file_out)

1st pair:  
`out_dir = f"{outputs_prefix}/S1_InSAR_2024-09-03-050122__2024-09-15-050123/sar`
2nd pair:  
`/data/res/flood-poland-2024/S1_InSAR_2024-09-04-165208__2024-09-16-165209/sar`

## Apply processing chains: coherence and change detection

In [ ]:
from eo_tools.S1.process import coherence, amplitude
from eo_tools.S1.process import apply_to_patterns_for_pair, apply_to_patterns_for_single
from pathlib import Path

# out_dir = f"{outputs_prefix}/S1_InSAR_2024-09-03-050122__2024-09-15-050123/sar"
geo_dir = Path(out_dir).parent

# compute interferometric coherence
# apply_to_patterns_for_pair(
#     coherence,
#     out_dir=out_dir,
#     file_prm_prefix="slc_prm",
#     file_sec_prefix="slc_sec",
#     file_out_prefix="coh",
#     box_size=[3, 10],
#     multilook=[1, 4],
# )

# compute primary amplitude
# apply_to_patterns_for_single(
#     amplitude,
#     out_dir=out_dir,
#     file_in_prefix="slc_prm",
#     file_out_prefix="amp_prm",
#     multilook=[2, 8],
# )

# compute secondary amplitude
# apply_to_patterns_for_single(
#     amplitude,
#     out_dir=out_dir,
#     file_in_prefix="slc_sec",
#     file_out_prefix="amp_sec",
#     multilook=[2, 8],
# )

# compute incoherent changes
apply_to_patterns_for_pair(
    change_detection,
    out_dir=out_dir,
    file_prm_prefix="amp_sec",
    file_sec_prefix="amp_prm",
    file_out_prefix="change_noflt",
)

## Apply geocoding, merge and crop subswaths

In [ ]:
from pathlib import Path

# out_dir = f"{outputs_prefix}/S1_InSAR_2024-09-03-050122__2024-09-15-050123/sar"
geo_dir = Path(out_dir).parent
from eo_tools.S1.process import geocode_and_merge_iw
# geocode_and_merge_iw(geo_dir, shp=shp, var_names=["change"], pol="vv", clip_to_shape=False)
# geocode_and_merge_iw(geo_dir, shp=shp, var_names=["coh","change", "amp_prm", "amp_sec"], clip_to_shape=False)
geocode_and_merge_iw(geo_dir, shp=shp, var_names=["change_noflt"], clip_to_shape=False)

## Visualize
Click on top right icon to hide/show layers

In [ ]:
from folium import TileLayer
m = folium.Map()

# _ = show_cog(f"{geo_dir}/coh_vv.tif", m, rescale="0,1")
# _ = show_cog(f"{geo_dir}/coh_vh.tif", m, rescale="0,1")
# _ = show_cog(f"{geo_dir}/amp_prm_vv.tif", m, rescale="0,1")
# _ = show_cog(f"{geo_dir}/amp_sec_vv.tif", m, rescale="0,1")
_ = show_cog(f"{geo_dir}/change_vv.tif", m, rescale="-0.5,-0.0", colormap_name="blues_r", resampling="bilinear")
_ = show_cog(f"{geo_dir}/change_flt_vv.tif", m, rescale="-0.5,-0.0", colormap_name="blues_r", resampling="bilinear")
_ = show_cog(f"{geo_dir}/change_noflt_vv.tif", m, rescale="-0.5,-0.0", colormap_name="blues_r", resampling="bilinear")
# _ = show_cog(f"{geo_dir}/change_noflt_vv.tif", m, rescale="0,1", colormap_name="blues_r", resampling="bilinear", expression="b1<-0.5")
# _ = show_cog(f"{geo_dir}/change_vh.tif", m, rescale="-1,0", colormap_name="blues_r")
# _ = show_cog(f"{geo_dir}/change_vh.tif", m, rescale="-0.25,0.25", colormap_name="rdbu_r")
LayerControl().add_to(m)
m

# Cropping a small window to adjust blf parameters

In [ ]:
# from eo_tools_dev.util import ttcog_get_stats
# ttcog_get_stats(f"{geo_dir}/change_vv.tif")

In [ ]:
from glob import glob
amp_files = glob(f"{out_dir}/amp*.tif")
img = riox.open_rasterio(amp_files[0])

In [ ]:
sub = img[0].copy()
# sub = img[0,-2000:-1000,-2000:-1000].copy()

In [ ]:
from ndsar import sarblf
from scipy.ndimage import median_filter
# flt = median_filter(sub.data.copy(), 2)
flt = sub.data.copy()
for _ in range(1):
    flt = sarblf(flt, 1.6, 0.5)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(flt, interpolation="none", vmin=0, vmax=.8, cmap="gray")
# plt.imshow(flt[600:, 600:], interpolation="none", vmin=0, vmax=.5, cmap="gray")
# plt.imshow(sub.data, interpolation="none", vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
# plt.imshow(flt, interpolation="none", vmin=0, vmax=1)
plt.imshow(sub.data, interpolation="none", vmin=0, vmax=.8, cmap="gray")
plt.colorbar(fraction=0.046, pad=0.04)